# **LAB-02 CPA**

### Integrantes:
- João Vitor Abitante - 21102407
- Luciano Farias - 22180074
- Eduardo Xavier - 19103394
- Alice Ferreira - 21101739



Para este trabalho, foi utilizado o dataset Iris em formato de CSV

In [ ]:
import requests
import pandas as pd
from pprint import pprint

In [ ]:
# Indice para fins de teste ao fazer os requests
index = 2

# **GET**

Podemos fazer um GET para obter tanto todos os dados como podemos fazer um GET para obter pelo Index

- **Buscando pelo Index**

    - Endpoint: http://127.0.0.1:5000/movie/index

In [ ]:
response = requests.get(f'http://127.0.0.1:5000/movie/{index}')
pprint(response.json())

- **Buscando todos os dados** 

    - Endpoint: http://127.0.0.1:5000/movie

In [ ]:
response = requests.get('http://127.0.0.1:5000/movie')
df = pd.DataFrame(response.json())
display(df.head())

# **Delete**

- Delete feito pelo Index
    - Endpoint: http://127.0.0.1:5000/movie/index

### Validação


- Antes de deletar

In [ ]:
response = requests.get(f'http://127.0.0.1:5000/movie/{index}')
print('Response antes de deletar:\n')
pprint(response.json())

- **DELETANDO**

In [ ]:
response = requests.delete(f'http://127.0.0.1:5000/movie/{index}')
print('Deletado com sucesso:', response.status_code) 

- Após deletar

In [ ]:
response = requests.get(f'http://127.0.0.1:5000/movie/{index}')
print('Response após deletar:\n')
pprint(response.json())

# **POST**

- Inserção de dados no CSV
    - Endpoint: http://127.0.0.1:5000/movie/index

### Validação

- CSV antes da inserção de novos dados

In [ ]:
response = requests.get(f'http://127.0.0.1:5000/movie')
df = pd.DataFrame(response.json())
display(df.tail())

- Inserindo novos dados com POST

In [ ]:
json={'Sepal_length': 5454, 'Sepal_width': 3.5, 'Petal_length': 1.4, 'Petal_width': 0.2, 'Species': 'Iris-setosa'}
requests.post('http://127.0.0.1:5000/movie', json=json)

- CSV após a inserção dos dados

In [ ]:
response = requests.get(f'http://127.0.0.1:5000/movie')
df = pd.DataFrame(response.json())
display(df.tail())

# **PUT**

- Atualização de determinado dado
    - Endpoint: http://127.0.0.1:5000/movie/index

### Validação

- Antes da atualização

In [ ]:
response = requests.get(f'http://127.0.0.1:5000/movie/{index}')
print('Response antes de atualizar:\n')
pprint(response.json())

- Executando o método PUT

In [ ]:
json = {'Sepal_length': 1111, 'Sepal_width': 2222, 'Petal_length': 1.4, 'Petal_width': 3.2, 'Species': 'Iris-setosa'}
requests.put(f'http://127.0.0.1:5000/movie/{index}', json=json)

- Após a atualização

In [ ]:
response = requests.get(f'http://127.0.0.1:5000/movie/{index}')
print('Response após atualizar:\n')
pprint(response.json())